In [3]:

import json
from datetime import date, datetime, timedelta
from time import time
import pytz



limit1 = datetime.strptime("9:00:00", "%H:%M:%S").time()
limit2 = datetime.strptime("20:00:00", "%H:%M:%S").time()
limit3 = datetime.strptime("12:00:00", "%H:%M:%S").time()



doctor_time_tables = {
    "Dr. João Santos": {"monday": {'start': '09:00:00', 'end': "13:00:00"}, "wednesday": {'start': '18:00:00','end': "20:00:00"}},
    "Dr. Miguel Costa": {"monday": {'start': "13:30:00", 'end': "17:30:00"}, "thursday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Sofia Pereira": {"monday": {'start': "18:00:00", 'end': "20:00:00"}, "thursday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dra. Mariana Chagas": {"tuesday": {'start': "09:00:00", 'end': "13:00:00"}, "thursday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. António Oliveira": {"tuesday": {'start': "13:30:00", 'end': "17:30:00"}, "friday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Inês Martins": {"tuesday": {'start': "18:00:00", 'end': "20:00:00"}, "friday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dr. Ângelo Rodrigues": {"wednesday": {'start': "09:00:00", 'end': "13:00:00"}, "friday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. José Dias": {"wednesday": {'start': "13:30:00", 'end': "17:30:00"}, "saturday": {'start': '09:00:00', 'end': "12:00:00"}},
}


In [4]:
def is_time_in_range(selected_time, time_list):
    selected_time = datetime.strptime(selected_time, "%H:%M:%S").time()
    
    for time in time_list:
        time_obj = datetime.strptime(time, "%H:%M:%S").time()
        if time_obj == selected_time:
            return True
    return False



In [5]:
def get_available_time_slots(doctor, day):
    # Function to get available time slots for a specific doctor on a given day
    doctor_schedule = doctor_time_tables.get(doctor, {}).get(day, {})
    
    if not doctor_schedule:
        return []

    start_time = datetime.strptime(doctor_schedule.get('start', '00:00:00'), "%H:%M:%S").time()
    end_time = datetime.strptime(doctor_schedule.get('end', '23:59:59'), "%H:%M:%S").time()

    available_times = []
    current_time = start_time

    while current_time <= end_time:
        available_times.append(current_time.strftime("%H:%M:%S"))
        current_time = (datetime.combine(date.today(), current_time) + timedelta(minutes=30)).time()

    return available_times

In [6]:
def get_available_times_message(doctor, day):
    available_times = get_available_time_slots(doctor, day)
    if not available_times:
        return f"{doctor} is not available on {day}."
    return f"{doctor} is available on {day} at the following times: {', '.join(available_times)}."


In [49]:
def choose_doctor_based_on_time(requested_time_str, requested_doctor, requested_date_str):
    available_doctors = []
    requested_date = datetime.strptime(requested_date_str, "%Y-%m-%d").strftime("%A").lower()   # Monday
    requested_time = datetime.strptime(requested_time_str, "%H:%M:%S").time()  # 09:00:00

    for doctor, schedules in doctor_time_tables.items():
        for day, time_range in schedules.items():
            if day == requested_date:
                start_time = datetime.strptime(time_range['start'], "%H:%M:%S").time()
                end_time = datetime.strptime(time_range['end'], "%H:%M:%S").time()

                if start_time <= requested_time <= end_time:
                    available_doctors.append(doctor)

    if not available_doctors:
        return None  # No available doctors for the requested time

    if requested_doctor in available_doctors:
        return requested_doctor

    return None  # The originally reques

In [ ]:
def doctor_checking(arguments):
    provided_date =  str(datetime.strptime(json.loads(arguments)['date'], "%Y-%m-%d").date())
    provided_time = str(datetime.strptime(json.loads(arguments)['time'].replace("PM","").replace("AM","").strip(), "%H:%M:%S").time())
    start_date_time = provided_date + " " + provided_time
    timezone = pytz.timezone('Europe/Lisbon')
    start_date_time = timezone.localize(datetime.strptime(start_date_time, "%Y-%m-%d %H:%M:%S"))
    email_address = json.loads(arguments)['email_address']
    doctor = json.loads(arguments)['doctor']
    end_date_time = start_date_time + timedelta(hours=0.5)

    
    existing_doctors = ["Dr. João Santos", "Dr. Miguel Costa", "Dra. Sofia Pereira", 'Dra. Mariana Chagas', 'Dr. António Oliveira',
                            'Dra. Inês Martins', 'Dr. Ângelo Rodrigues', 'Dr. José Dias']


    if doctor is not None:
        if doctor not in existing_doctors:
            return "Please choose a valid doctor from the list: " + ", ".join(existing_doctors)
                
        day_of_week = start_date_time.strftime("%A").lower()
        doctor_schedule = get_available_time_slots(doctor, day_of_week)

        if not doctor_schedule or not is_time_in_range(provided_time, doctor_schedule):
            available_times_message = get_available_times_message(doctor, day_of_week)
            
            return f"{doctor} is not available at the selected time. {available_times_message} Please choose a valid time."
        else:
            # Choose a doctor based on the requested time
            requested_time = start_date_time.time().strftime("%H:%M:%S")
            updated_doctor = choose_doctor_based_on_time(requested_time, doctor, provided_date)
            if updated_doctor is None:
                return False
            # Update the 'doctor' variable with the chosen doctor
            #doctor = updated_doctor
            # Confirm the appointment
            return True



In [12]:
doctor_schedule = get_available_time_slots('Dr. João Santos', 'monday')

In [13]:
is_time_in_range('17:00:00', doctor_schedule)

False

In [18]:
timezone = pytz.timezone('Europe/Lisbon')
doctor = 'Dr. João Santos'
provided_time = '17:00:00'
provided_date = datetime.strptime('2024-01-01', "%Y-%m-%d").date()
start_date_time = datetime.combine(provided_date, datetime.strptime(provided_time, "%H:%M:%S").time())
start_date_time = timezone.localize(start_date_time)


if not doctor_schedule or not is_time_in_range(provided_time, doctor_schedule):
        available_times_message = get_available_times_message(doctor, 'monday')
        print(f"{doctor} is not available at the selected time. {available_times_message} Please choose a valid time.")
else:
        # Choose a doctor based on the requested time
        requested_time = start_date_time.time().strftime("%H:%M:%S")
        updated_doctor = choose_doctor_based_on_time(requested_time, doctor, provided_date)
        if updated_doctor is None:
                print(False)

Dr. João Santos is not available at the selected time. Dr. João Santos is available on monday at the following times: 09:00:00, 09:30:00, 10:00:00, 10:30:00, 11:00:00, 11:30:00, 12:00:00, 12:30:00, 13:00:00. Please choose a valid time.


In [ ]:
is_time_in_range()

In [50]:
updated_doctor = choose_doctor_based_on_time('09:00:00', 'Dr. João Santos', '2024-01-01')
print(updated_doctor)

monday
Dr. João Santos
{'start': '09:00:00', 'end': '13:00:00'}
Dr. João Santos


In [2]:
import json
from datetime import date, datetime, timedelta
from time import time
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import pytz
from util import local_settings

ModuleNotFoundError: No module named 'googleapiclient'

In [6]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

ModuleNotFoundError: No module named 'google'

In [2]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

timezone = pytz.timezone('Europe/Lisbon')
doctor = 'Dr. João Santos'
provided_time = '13:00:00'
provided_date = datetime.strptime('2024-01-01', "%Y-%m-%d").date()
start_date_time = datetime.combine(provided_date, datetime.strptime(provided_time, "%H:%M:%S").time())
start_date_time = timezone.localize(start_date_time)
email_address = 'catioli@hotmail.com'


if start_date_time.time() >= limit1 and start_date_time.time() <= limit2: #Day of the week
                        end_date_time = start_date_time + timedelta(hours=2)
                        events = service.events().list(calendarId="primary").execute()
                        id = ""
                        final_event = None
                        for event in events['items']:
                            if event['attendees'][0]['email'] == email_address:
                                id = event['id']
                                final_event = event


ModuleNotFoundError: No module named 'googleapiclient'